In [1]:
import pandas as pd
import math

In [181]:
#Evaluates a sample using the given decision tree dt. Returns true if the decision tree gets it right, false if not
def evalSample(dt, sample):   
    while (len(dt) == 2):
        dt = dt[1][sample[dt[0]]]
    return (dt[0] == sample['party'])

infileTrain = pd.read_csv("train.csv") #training set
infilePrune = pd.read_csv("prune.csv") #prune set
infileTest = pd.read_csv("test.csv") #testing set


In [182]:
#Just a dummy assingment
df = infileTrain 
#Dummy Decision Tree, ID3 should create your own decision tree.This is a nested representation. Each node is a list. 
#if the node has one entry then it is a leaf node and performs the classification. Otherwise the node has two entries:
#the first entry is the feature ('bruises') and the second entry is a dictionary containing the tree below this node
#for example, bruises has two different possible values ('f' and 't'), therefore the dictionary has those two values
#'t' goes to a leaf node classifying 'p'. However, the 'f' branch has more decision making in this case 'gill-spacing' 
#that all lead to terminal nodes.  
dp = infilePrune
ddt = infileTest

In [186]:
df

,party,handicappedInfants,waterProject,budgetResolution,physicianFeeFreeze,elSalvadorAid,religiousGroupsInSchools,antiSatelliteTestBan,aidToNicaraguanContras,mxMissile,immigration,synfuelsCorporationCutback,educationSpending,superfundRightToSue,crime,dutyFreeExports,exportsSouthAfrica
0,republican,n,y,n,y,y,y,n,n,n,y,noVote,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,noVote
2,democrat,noVote,y,y,noVote,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,noVote,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,noVote,y,y,y,y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,republican,n,n,n,y,y,y,n,n,n,y,noVote,y,y,y,n,n
100,democrat,n,n,y,n,n,y,y,noVote,y,y,y,n,n,n,y,y
101,democrat,y,y,noVote,y,y,y,n,n,y,n,y,noVote,y,y,n,n
102,democrat,noVote,y,n,n,n,n,y,y,y,y,y,n,n,y,y,y


In [8]:
from math import log

#compute entropy
def entropy(examples):
    numAttributes = len(examples)
    featureCounts ={}
    for example in examples:
        currentLable=example[-1] 
        if currentLable not in featureCounts.keys():
            featureCounts[currentLable]=0
        featureCounts[currentLable]+=1
    result=0.0
    for i in featureCounts:
        p = float(featureCounts[i]) / numAttributes
        result -= p * log(p, 2)
    return result


def splitDataSet(examples,feat,value):
    subSet=[]
    #return the subset that does not contain values in traget feature
    for featVec in examples:
        #print(featVec[axis])
        if featVec[feat]==value : 
            reduced=featVec[:feat]
            reduced.extend(featVec[feat+1:])
            subSet.append(reduced)
    return subSet 


def BestFeature(examples):
    num=len(examples[0])-1
    print(num)
    bestFeature=-1
    InfoGain=0
    #H(S)
    HEntropy=entropy(examples)
    for i in range(num):
        featList=[example[i] for example in examples] #GET VALUES AT FRETURE[i]
        uniqlVals=set(featList) #SET UNIQUE VALUES
        cEntropy=0
        for value in uniqlVals:
            subDataSet=splitDataSet(examples,i,value)
            prob=len(subDataSet)/float(len(examples)) #p(t)
            cEntropy+=prob*entropy(subDataSet) #H(S/A)    
        infoGain=HEntropy-cEntropy #IG
        #print(infoGain)
        if (infoGain>InfoGain):
            InfoGain=infoGain
            bestFeature=i
    return bestFeature 

In [9]:
def createDataSet(example):
    #rearrange the dataframe and convert to list
    data = example[[c for c in df if c not in ['party']] + ['party']]
    dataSet= data.values.tolist()
    labels= data.columns.tolist()
    return dataSet,labels

In [10]:
def ID3(dataSet,Feature):
    #If all examples are in the same class
    classList=[example[-1] for example in dataSet]
    #print(dataSet[0])
    if classList.count(classList[-1])==len(classList):
        return classList[-1]
    #If Examples is empty,Then below this new branch return most common target value in the examples
    if len(dataSet[0]) == 1:
        return max(set(classList), key = classList.count)  
    bestFeat=BestFeature(dataSet)
    #print(bestFeat,Feature)
    bestFeatLabel=Feature[bestFeat] 
    #print(bestFeatLabel)
    Tree={bestFeatLabel:{}} 
    #print(Feature[bestFeat]) remove the feature from the feature list
    Feature.remove(bestFeatLabel)
    subLabels=Feature[:]
    #print(subLabels)
    featValues=[example[bestFeat] for example in dataSet]
    uniqVals=set(featValues)
    ##print(uniqueVals)
    #For each possible value of A, Add a new tree branch below
    for value in uniqVals:
        Tree[bestFeatLabel][value]=ID3(splitDataSet(dataSet,bestFeat,value),subLabels)
    return Tree

In [160]:
v = []
b= a.get('physicianFeeFreeze')
c = b.get('y')
#sum(len(b) for b in b.itervalues())
party = ['democrat','republican']
values = ['noVote','n','y']
b.get(values[2])
v.append(c.copy())
d = c.get('budgetResolution').get('n')
v.append(d.copy())
c
idt3 = {'physicianFeeFreeze': {'noVote': 'democrat',
  'n': 'democrat',
  'y': {'budgetResolution': {'noVote': 'democrat',
    'n': 'republican',
    'y': 'republican'}}}}

In [ ]:
def split(examples,feat):
    subSet=[]
    #return the subset that does not contain values in traget feature
    for featVec in examples:
        #print(featVec[axis]):
        reduced=featVec[:feat]
        reduced.extend(featVec[feat+1:])
        subSet.append(reduced)
    return subSet 

In [155]:
def prune(tree,Feature,dataSet):
    temp = []
    temp_fe = []
    party = ['democrat','republican']
    values = ['noVote','n','y']
    feat = Feature #list of features
    newtree = []
    #iterate throught the feature list as the node labels
    for features in Feature:
        #temp.append(tree.get(features).copy())
        #check if the branches after this node are leafs
        if tree.get(features) != None:
            for v in values:
                if tree.get(values[v]) not in party: #chech if the node is leafs
                    temp.append(tree.get(values[v]).copy()) #store none leaf branches at temp starting from root
                    temp_fe.append(features)
    numnode = len(temp_fea)
    for i in range(numnode):
    #get the vote from the smallest braches
        dataset = split(dataSet,temp_fe[len(temp_fe)-1])
        classList=[example[-1] for example in dataset]
        vote = max(set(classList), key = classList.count)
    #replace the node with vote num
    #newtree = 
    #compute for accuracy at all node
        beforepruning = accuracy(tree, infilePrune) 
        beforepruning = accuracy(newtree, infilePrune) 
        if afterpruning >= beforepruning:
            newtree = tree
    return newtree


In [156]:
#classifier based on genreated decision tree and validation set
def classify(idt,Labels,testVec):
    classLabel = {}
    #get first attribute
    firstAttribute=list(idt.keys())[0]
    #create sec dic
    secondDict=idt[firstAttribute] 
    #get index
    Index=Labels.index(firstAttribute) 
    for key in secondDict.keys():
        if testVec[Index]==key:
            if type(secondDict[key]).__name__=='dict':
                classLabel=classify(secondDict[key],Labels,testVec)
            else:classLabel=secondDict[key]
    return classLabel

#Get prediction results from idt and validation set
def predict(idt,testdata):
    testdata, testlabels = createDataSet(testdata)
    #classify(idt,testlabels,testdata)
    classes = []
    for i in range(len(testdata)):
        a = classify(idt,testlabels,testdata[i])
        classes.append(a)
    return classes
#compute the accuracy of predictions
def accuracy(idt,val):
    res = predict(idt,val)
    validation = val.iloc[:,0].values.tolist()
    count = 0
    for i in range(len(res)):
        #print(res[i]+' '+validation[i])
        if res[i] == validation[i]:
            count += 1
    return count/len(res)
    

In [161]:
#accuracy after pruning
accuracy(newtree,infileTest)

0.9328358208955224

In [162]:
accuracy(newtree,infileTrain)

0.9807692307692307

In [174]:
accuracy(newtree,infilePrune)

0.949238578680203

In [178]:
idt ={'physicianFeeFreeze': {'noVote': 'democrat', 'n': 'democrat', 'y': {'budgetResolution': {'noVote': 'democrat', 'n': {'dutyFreeExports': {'noVote': 'republican', 'n': 'republican', 'y': 'democrat'}}, 'y': {'handicappedInfants': {'noVote': 'republican', 'n': 'democrat', 'y': 'republican'}}}}}}

In [179]:
accuracy(idt,infilePrune)

0.9289340101522843